## Ultimate application


In [ ]:
# ===============================================================
# Cell 1: 全局导入与设置 (最终版)
# ===============================================================
import os
import sys
import json
import uuid
import nest_asyncio
import asyncio
import httpx
import uvicorn
import threading
import datetime
from functools import wraps
from contextlib import asynccontextmanager
from dataclasses import dataclass, field
from typing import List, Dict, Literal, Callable

# --------------------------------------------------------------
# 🧩 关键：确保本地项目路径正确
# --------------------------------------------------------------
# 请将此路径替换为您项目中 `python` 目录的实际绝对路径
# 例如： 'E:\\Github\\a2a-samples\\samples\\python'
# 或者 'e:/Github/a2a-multiagent-host-demo/demo/ui'
# 总之，这个路径下应该能找到 pages, components, state 等目录
local_project_path = r"e:\Github\a2a-multiagent-host-demo\demo\ui"
if local_project_path not in sys.path:
    print(f"Adding to sys.path: {local_project_path}")
    sys.path.insert(0, local_project_path)

# --------------------------------------------------------------
# 🧩 关键：导入所有必要的库和本地模块
# --------------------------------------------------------------
import mesop as me
from fastapi import FastAPI
from fastapi.middleware.wsgi import WSGIMiddleware
from dotenv import load_dotenv
from mesop.components.select.select import SelectOption

from state.state import AppState, ChatMessage, AgentTask

from pages import conversation as conversation_page_module
from components.conversation_list import conversation_list

# 允许在Jupyter环境中重用asyncio事件循环并加载环境变量
nest_asyncio.apply()
load_dotenv()

# 全局配置
OLLAMA_HOST = os.getenv('OLLAMA_HOST', 'http://localhost:11434')
DEFAULT_MODEL = "huanhuan-qwen" # 您可以换成自己喜欢的默认模型

# 全局启动数据，用于在应用启动时一次性获取数据
STARTUP_DATA = {
    "ollama_connected": False,
    "available_models": [],
}

# 全局服务实例占位符
ollama_service: "OllamaService"
task_manager: "TaskManager"
security_manager: "SecurityManager"
auth_service: "AuthService"

print("✅ Cell 1: Imports and Global Setup complete.")

Adding to sys.path: e:\Github\a2a-multiagent-host-demo\demo\ui
✅ Cell 1: Imports and Global Setup complete.


## cell2

In [ ]:
# ===============================================================
# Cell 2: 核心服务定义 (最终简化版 - 已移除 TaskManager)
# ===============================================================
# TaskManager 类已被完全移除，因为其逻辑已整合进 pages/conversation.py 的事件处理器中。

class SecurityManager:
    """中心化的安全配置与审计日志管理器。"""
    def __init__(self):
        self.policy = me.SecurityPolicy(allowed_script_srcs=['https://cdn.jsdelivr.net'])
        self.audit_log = []
    def log_event(self, event_type: str, details: str):
        timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        log_entry = f"[{timestamp}] - {event_type}: {details}"
        self.audit_log.append(log_entry)
        print(f"AUDIT LOG: {log_entry}")

security_manager = SecurityManager()

class AuthService:
    """权限管理服务，与安全管理器集成。"""
    def __init__(self, security_manager_instance: SecurityManager):
        # 任务页面(tasks)仍然保留，以显示由旧版代理框架可能创建的任务
        self._user_permissions = {"guest": ["chat"], "admin": ["chat", "tasks", "audit"]}
        self.current_user_role = "guest"
        self.security_manager = security_manager_instance
    def check_permission(self, page_key: str) -> bool:
        return page_key in self._user_permissions.get(self.current_user_role, [])
    def set_user_role(self, role: str):
        if role in self._user_permissions and role != self.current_user_role:
            old_role, self.current_user_role = self.current_user_role, role
            self.security_manager.log_event("ROLE_CHANGE_SUCCESS", f"User role changed from '{old_role}' to '{self.current_user_role}'")
        elif role not in self._user_permissions:
            self.security_manager.log_event("ROLE_CHANGE_FAILURE", f"Attempted to change to an invalid role: '{role}'")

auth_service = AuthService(security_manager)

class OllamaService:
    """与 Ollama API 交互的服务，负责流式聊天。"""
    def __init__(self, client: httpx.AsyncClient):
        self.async_client = client
    async def check_connection(self) -> bool:
        try:
            r = await self.async_client.get(f"{OLLAMA_HOST}/api/tags", timeout=5); return r.status_code == 200
        except httpx.RequestError: return False
    async def get_available_models(self) -> List[str]:
        if not await self.check_connection(): return []
        try:
            r = await self.async_client.get(f"{OLLAMA_HOST}/api/tags")
            if r.status_code == 200: return [m['name'] for m in r.json().get('models', [])]
        except Exception as e: print(f"获取模型列表失败: {e}"); return []
    def stream_chat(self, model: str, messages: List[Dict], options: Dict):
        """返回一个异步生成器，用于流式传输聊天响应。"""
        url, payload = f"{OLLAMA_HOST}/api/chat", {"model": model, "messages": messages, "stream": True, "options": options}
        async def event_stream():
            try:
                async with self.async_client.stream("POST", url, json=payload, timeout=60) as r:
                    r.raise_for_status()
                    async for line in r.aiter_lines():
                        if line:
                            try:
                                data = json.loads(line)
                                if data.get('done') is False:
                                    yield data.get('message', {}).get('content', '')
                            except json.JSONDecodeError: continue
            except Exception as e:
                yield f"错误: {e}"
        return event_stream()


print("✅ Cell 2: Core Service definitions complete. ")

✅ Cell 2: Core Service definitions complete. TaskManager has been removed.


## cell3

In [3]:
# ==============================================================================
# Cell 3: UI组件、页面定义与事件处理器 (最终修正版)
# ==============================================================================

# --- 全局事件处理器 ---
def on_model_select(e: me.SelectSelectionChangeEvent): me.state(AppState).selected_model = e.value
def on_temperature_change(e: me.SliderValueChangeEvent): me.state(AppState).temperature = e.value
def on_top_p_change(e: me.SliderValueChangeEvent): me.state(AppState).top_p = e.value
def on_top_k_change(e: me.SliderValueChangeEvent): me.state(AppState).top_k = e.value

def on_clear_chat(e: me.ClickEvent):
    state = me.state(AppState)
    state.messages = []
    state.user_input = ""
    security_manager.log_event("CHAT_CLEAR", f"User '{auth_service.current_user_role}' cleared the chat history.")

def on_load_main_page(e: me.LoadEvent):
    """主页面加载时执行的函数。"""
    state = me.state(AppState)
    if not state.is_initialized:
        state.ollama_connected = STARTUP_DATA["ollama_connected"]
        state.available_models = STARTUP_DATA["available_models"]
        if state.ollama_connected and state.available_models:
            state.selected_model = state.available_models[0]
        state.is_initialized = True
    me.set_theme_mode("system")

# --- UI 组件定义 ---
def ui_sidebar():
    """渲染侧边栏，包含对话列表和模型设置。"""
    state = me.state(AppState)
    with me.box(style=me.Style(
        width=320, height="100vh",
        border=me.Border(right=me.BorderSide(style="solid", width=1, color=me.theme_var("outline-variant"))),
        display="flex", flex_direction="column", flex_shrink=0,
    )):
        with me.box(style=me.Style(padding=me.Padding.all(16), flex_shrink=0)):
             me.text("Ollama & Agents", type="headline-6")
        with me.box(style=me.Style(padding=me.Padding.symmetric(horizontal=16), flex_shrink=0)):
            conversation_list()
        
        with me.box(style=me.Style(padding=me.Padding.symmetric(vertical=16))):
            me.divider()

        with me.box(style=me.Style(
            padding=me.Padding.symmetric(horizontal=16),
            flex_grow=1, overflow_y="auto",
        )):
            me.text("⚙️ 设置", type="subtitle-1")
            with me.box(style=me.Style(display="flex", align_items="center", margin=me.Margin.symmetric(vertical=8))):
                me.icon("check_circle" if state.ollama_connected else "error", style=me.Style(color="green" if state.ollama_connected else "red"))
                me.text(f"Ollama {'已连接' if state.ollama_connected else '未连接'}", style=me.Style(margin=me.Margin.symmetric(horizontal=8)))
            
            me.text("🤖 模型", type="body-2", style=me.Style(margin=me.Margin(top=16), color=me.theme_var("on-surface-variant")))
            if state.ollama_connected and state.available_models:
                model_options = [SelectOption(value=model_name, label=model_name) for model_name in state.available_models]
                me.select(options=model_options, value=state.selected_model, on_selection_change=on_model_select, style=me.Style(width="100%"))
            else:
                me.text("未找到可用模型。")
                
            me.text("🎛️ 参数", type="body-2", style=me.Style(margin=me.Margin(top=24), color=me.theme_var("on-surface-variant")))
            me.text("Temperature", style=me.Style(font_size=14))
            me.slider(min=0.1, max=2.0, step=0.1, value=state.temperature, on_value_change=on_temperature_change)
            me.text("Top P", style=me.Style(font_size=14))
            me.slider(min=0.1, max=1.0, step=0.1, value=state.top_p, on_value_change=on_top_p_change)
            me.text("Top K", style=me.Style(font_size=14))
            me.slider(min=1, max=100, step=1, value=state.top_k, on_value_change=on_top_k_change)

        with me.box(style=me.Style(padding=me.Padding.all(16), flex_shrink=0)):
            me.button("🗑️ 清空当前对话", on_click=on_clear_chat, type="stroked", style=me.Style(width="100%"))

# --- 页面函数定义 ---

@me.content_component
def page_scaffold(title: str):
    """
    一个通用的页面骨架，用于包裹其他页面内容。
    使用 @me.content_component 确保它能正确地与 'with' 语句和 'me.slot()' 一起工作。
    """
    with me.box(style=me.Style(padding=me.Padding.all(24), width="100%")):
        me.text(title, type="headline-4")
        with me.box(style=me.Style(margin=me.Margin(top=16))):
            me.slot()

def main_chat_page():
    """主聊天页面，采用 flex 布局。"""
    with me.box(style=me.Style(display="flex", flex_direction="row", height="100vh")):
        ui_sidebar()
        with me.box(style=me.Style(flex_grow=1, display="flex")):
             conversation_page_module.conversation_page(me.state(AppState))

def tasks_dashboard_page():
    """代理任务仪表盘页面。"""
    state = me.state(AppState)
    with page_scaffold("代理任务仪表盘"):
        if not state.tasks: me.text("尚未启动任何任务."); return
        for task in sorted(state.tasks.values(), key=lambda t: t.start_time, reverse=True):
            with me.card(style=me.Style(margin=me.Margin.symmetric(vertical=8))):
                with me.box(style=me.Style(padding=me.Padding.all(16))):
                    me.text(f"任务 ID: {task.task_id[:8]}...", style=me.Style(font_weight="bold"))
                    me.text(f"状态: {task.status}", style=me.Style(font_family="monospace"))
                    me.text(f"提示: {task.prompt}")
                    with me.box(style=me.Style(max_height=200, overflow_y="auto", background="#fafafa", padding=me.Padding.all(8), border_radius=4, margin=me.Margin(top=8))):
                        me.markdown(task.result if task.result else "等待响应中...")

def audit_page():
    """安全审计日志页面。"""
    # ==========================================================
    # 关键修复：补全 'with page_scaffold(...):' 这一行
    # ==========================================================
    with page_scaffold("安全审计日志"):
        if not security_manager.audit_log: me.text("没有记录到审计事件。")
        else:
            for entry in reversed(security_manager.audit_log): me.text(entry, style=me.Style(font_family="monospace", white_space="pre-wrap"))

# --- 页面注册表 ---
ALL_PAGES = [
    {"path": "/", "title": "Ollama & Agents", "page_key": "chat", "on_load": on_load_main_page, "func": main_chat_page},
    {"path": "/tasks", "title": "任务仪表盘", "page_key": "tasks", "on_load": None, "func": tasks_dashboard_page},
    {"path": "/audit", "title": "审计日志", "page_key": "audit", "on_load": None, "func": audit_page},
]

print("✅ Cell 3: UI Components and Page definitions complete. SyntaxError in audit_page has been fixed.")

✅ Cell 3: UI Components and Page definitions complete. SyntaxError in audit_page has been fixed.


## cell4

In [ ]:
# ===============================================================
# Cell 4: 应用生命周期与启动 (最终简化版 - 已移除 TaskManager)
# ===============================================================

@asynccontextmanager
async def lifespan(app: FastAPI):
    """FastAPI 应用的生命周期钩子函数，负责应用的初始化和清理。"""
    # task_manager 已被移除，不再需要声明为全局变量
    global ollama_service
    
    async with httpx.AsyncClient(timeout=30) as client:
        # 步骤 1: 初始化核心服务
        ollama_service = OllamaService(client)

        # 步骤 2: 依赖注入
        conversation_page_module.ollama_service = ollama_service
        conversation_page_module.security_manager = security_manager
        conversation_page_module.auth_service = auth_service
        print("✅ 核心服务已初始化并注入到页面模块。")

        # 步骤 3: 执行一次性启动检查
        print("正在检查Ollama连接...")
        STARTUP_DATA["ollama_connected"] = await ollama_service.check_connection()
        if STARTUP_DATA["ollama_connected"]:
            print("Ollama连接成功。正在获取模型列表...")
            STARTUP_DATA["available_models"] = await ollama_service.get_available_models()
            print(f"找到模型: {STARTUP_DATA['available_models']}")
        else:
            print("警告: 无法连接到Ollama服务。")

        # 步骤 4: 动态注册所有页面
        for page_def in ALL_PAGES:
            def create_wrapper(p_def):
                @wraps(p_def["func"])
                def wrapper():
                    if not auth_service.check_permission(p_def["page_key"]):
                        security_manager.log_event("ACCESS_DENIED", f"User '{auth_service.current_user_role}' denied access to '{p_def['page_key']}'")
                        me.text(f"访问被拒绝。您没有权限查看 {p_def['title']} 页面。")
                        return
                    security_manager.log_event("ACCESS_GRANTED", f"User '{auth_service.current_user_role}' granted access to '{p_def['page_key']}'")
                    p_def["func"]()
                return wrapper
            
            me.page(
                path=page_def["path"],
                title=page_def["title"],
                security_policy=security_manager.policy,
                on_load=page_def["on_load"]
            )(create_wrapper(page_def))
        
        # 步骤 5: 创建并挂载 Mesop 应用
        mesop_app = me.create_wsgi_app(debug_mode=False)
        app.mount("/", WSGIMiddleware(mesop_app))
        
        print("应用启动完成，所有页面已注册。")
        yield
    
    print("应用关闭，正在清理资源。")

def start_app():
    """启动应用的函数。"""
    auth_service.set_user_role("admin")
    print(f"当前用户角色设置为: '{auth_service.current_user_role}'")
    
    app = FastAPI(lifespan=lifespan)
    host = os.environ.get('A2A_UI_HOST', '127.0.0.1')
    port = int(os.environ.get('A2A_UI_PORT', '12000'))
    
    def run_uvicorn():
        try:
            uvicorn.run(app, host=host, port=port, log_level="info")
        except Exception as e:
            print(f"启动Uvicorn时出错: {e}")

    if not any(t.name == 'UvicornThread' for t in threading.enumerate()):
        uvicorn_thread = threading.Thread(target=run_uvicorn, name='UvicornThread', daemon=True)
        uvicorn_thread.start()
        print(f"✅ Uvicorn 已在后台启动： http://{host}:{port}")
        print("请在浏览器中打开以上地址访问应用。")
        if auth_service.current_user_role == "admin":
            # 我们仍然保留 /tasks 页面的链接，因为它可能由旧的代理框架使用
            print(f"   - 任务仪表盘: http://{host}:{port}/tasks")
            print(f"   - 审计日志: http://{host}:{port}/audit")
    else:
        print("ℹ️ Uvicorn 似乎已在运行中。")

# --- 运行应用 ---
start_app()

AUDIT LOG: [2025-07-30 20:35:56] - ROLE_CHANGE_SUCCESS: User role changed from 'guest' to 'admin'
当前用户角色设置为: 'admin'
✅ Uvicorn 已在后台启动： http://127.0.0.1:12000
请在浏览器中打开以上地址访问应用。
   - 任务仪表盘: http://127.0.0.1:12000/tasks
   - 审计日志: http://127.0.0.1:12000/audit


INFO:     Started server process [9796]
INFO:     Waiting for application startup.


✅ 核心服务已初始化并注入到页面模块。
正在检查Ollama连接...
Ollama连接成功。正在获取模型列表...


INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:12000 (Press CTRL+C to quit)


找到模型: ['huanhuan:latest', 'qwen2.5:0.5b', 'qwen2.5:3b']
应用启动完成，所有页面已注册。
INFO:     127.0.0.1:14073 - "GET / HTTP/1.1" 200 OK
INFO:     127.0.0.1:14073 - "GET /styles.css HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:14074 - "GET /zone.js/bundles/zone.umd.js HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:14075 - "GET /prod_bundle.js HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:14075 - "POST /__ui__ HTTP/1.1" 200 OK
AUDIT LOG: [2025-07-30 20:36:01] - ACCESS_GRANTED: User 'admin' granted access to 'chat'
INFO:     127.0.0.1:14091 - "POST /__ui__ HTTP/1.1" 200 OK
AUDIT LOG: [2025-07-30 20:36:09] - ACCESS_GRANTED: User 'admin' granted access to 'chat'
AUDIT LOG: [2025-07-30 20:36:09] - ACCESS_GRANTED: User 'admin' granted access to 'chat'
INFO:     127.0.0.1:14091 - "POST /__ui__ HTTP/1.1" 200 OK
AUDIT LOG: [2025-07-30 20:36:11] - ACCESS_GRANTED: User 'admin' granted access to 'chat'
AUDIT LOG: [2025-07-30 20:36:11] - ACCESS_GRANTED: User 'admin' granted access to 'chat'
AUDIT LOG: [2025-